In [ ]:
# Possible Inputs

# 1. 
# JSON file

# 2. 
# Do you want to create GA4 Config Tag
# if Yes; Specify Measurement ID (Use any dummy value to create temporary Tag)
# if No; Specify Tag Name 


# Notes

# Event name

In [ ]:
import os
import json
import pandas as pd
import time
import requests
import re

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GTM-NLDQ2CR_workspace5.json to GTM-NLDQ2CR_workspace5.json


In [ ]:
def formatString(q):
  result = re.search('{{(.+?)}}', q)
  if result:
      result = result.group(1).replace(" ", "_").lower()
      return result
  else:
    return q

def extractString(q):
  result = re.search('{{(.+?)}}', q)
  if result:
      result = result.group(1)
      return result
  else:
    return q

def setEventParams(name, value):
  return {
    "type": "MAP",
    "map": [
      { "type": "TEMPLATE", "key": "name", "value": name },
      { "type": "TEMPLATE", "key": "value", "value": value }
    ]
  }

In [ ]:
def createGA4Config(json_data, tagId, measurementId, fingerprint):

  return {
    "accountId": json_data['containerVersion']['accountId'],
    "containerId": json_data['containerVersion']['containerId'],
    "tagId": str(tagId),
    "name": "GA4 Configuration",
    "type": "gaawc",
    "parameter": [
      { "type": "BOOLEAN", "key": "sendPageView", "value": "false" },
      { "type": "BOOLEAN", "key": "enableSendToServerContainer", "value": "false" },
      { "type": "TEMPLATE", "key": "measurementId", "value": measurementId },
    ],
    "fingerprint": fingerprint,
    "firingTriggerId": ["2147479553"],
    "tagFiringOption": "ONCE_PER_EVENT",
    "monitoringMetadata": { "type": "MAP" },
    "consentSettings": { "consentStatus": "NOT_SET" },
  }

In [ ]:
def createGA4PVParams(measurementId):

  return [
        {
            "type": "TEMPLATE",
            "key": "eventName",
            "value": "page_view"
        },
        {
            "type": "LIST",
            "key": "eventParameters",
            "list": [
                {
                    "type": "MAP",
                    "map": [
                        {
                            "type": "TEMPLATE",
                            "key": "name",
                            "value": "location"
                        },
                        {
                            "type": "TEMPLATE",
                            "key": "value",
                            "value": "{{Page URL}}"
                        }
                    ]
                }
            ]
        },
        {
            "type": "TAG_REFERENCE",
            "key": "measurementId",
            "value": measurementId
        }
    ]

In [ ]:
def setGA4Params(tag_parameters, eventName, measurementId, extraParams):

  eventCategory = None
  eventAction = None
  eventLabel = None
  eventValue = None
  dimension = None
  metric = None

  if(tag_parameters):
    for tag_parameter in tag_parameters:
      if(tag_parameter["key"] == "trackType" and tag_parameter["value"] == "TRACK_PAGEVIEW"):
        return "pageview"
      if(tag_parameter["key"] == "eventCategory"):
        eventCategory = tag_parameter["value"]
      elif(tag_parameter["key"] == "eventAction"):
        eventAction = tag_parameter["value"]
      elif(tag_parameter["key"] == "eventLabel"):
        eventLabel = tag_parameter["value"]
      elif(tag_parameter["key"] == "eventValue"):
        eventValue = tag_parameter["value"]
      elif(tag_parameter["key"] == "dimension"):
        dimension = tag_parameter["list"]
      elif(tag_parameter["key"] == "metric"):
        metric = tag_parameter["list"]

  param_list = [
                setEventParams("eventCategory", eventCategory) if (eventCategory) else None,
                setEventParams("eventAction", eventAction) if (eventAction) else None,
                setEventParams("eventLabel", eventLabel) if (eventLabel) else None,
                setEventParams("eventValue", eventValue) if (eventValue) else None
  ]

  if(dimension):
    for i in dimension:
      param_list.append(setEventParams(formatString(i["map"][1]["value"]), i["map"][1]["value"]))

  if(metric):
    for i in metric:
      param_list.append(setEventParams(formatString(i["map"][1]["value"]), i["map"][1]["value"]))

  if(extraParams):
    param_list.extend(extraParams)

  return [
    { "type": "TEMPLATE", "key": "eventName", "value": eventName },
    {
      "type": "LIST",
      "key": "eventParameters",
      "list": list(filter(None, param_list)),
    },
    { "type": "TAG_REFERENCE", "key": "measurementId", "value": measurementId },
  ]

In [ ]:
def extractParamsFromVar(var):
  params = var.get('parameter')
  param_list = []
  if(params):
    for param in params:
      if(param["key"] == "dimension"):
        dimension = param["list"]
      elif(param["key"] == "metric"):
        metric = param["list"]

    if(dimension):
      for i in dimension:
        param_list.append(setEventParams(formatString(i["map"][1]["value"]), i["map"][1]["value"]))

    if(metric):
      for i in metric:
        param_list.append(setEventParams(formatString(i["map"][1]["value"]), i["map"][1]["value"]))
        
  else:
    param_list = None

  return param_list

In [ ]:
fingerprint = round(time.time() * 1000)

def covertUAToGA4(filename):
  global fingerprint
  importJSON = {}
  ga4_tags = []
  maxTagID = 0
  measurementId = "G-A2ABC2ABCD"
  measurementTagName = "GA4 Configuration"      # GA4 Configuration tag name
  eventName = "GA4_Event"     # To be changed
  tag_paused = False

  json_data = json.loads(open('{0}'.format(filename)).read())
  tags = json_data['containerVersion']['tag'] if('tag' in json_data['containerVersion']) else {}
  vars = json_data['containerVersion']['variable'] if('variable' in json_data['containerVersion']) else {}

  maxTagID = int(max(tags, key=lambda x: int(x.get('tagId'))).get('tagId'))

  ga4_tags.extend(tags)
  ga4_tags.append(createGA4Config(json_data, maxTagID+1, measurementId, fingerprint))
  maxTagID += 1

  for tag in tags:
    if("paused" in tag and tag["paused"]):
      tag_paused = True
    if(tag.get('type') == "ua" and not tag_paused):
      fingerprint = round(time.time() * 1000)

      ga4_tag = tag.copy()
      ga4_tag['tagId'] = str(maxTagID+1)
      ga4_tag['name'] = 'GA4 {0}'.format(tag['name'])
      ga4_tag['type'] = 'gaawe'
      ga4_tag['fingerprint'] = fingerprint

      gaSettings = ""

      if('parameter' in tag):
        parameter = tag.get('parameter')
        if(parameter):
          for i in parameter:
            if(i['key'] == "gaSettings"):
              varName = extractString(i['value'])
              for var in vars:
                if(var['name'] == varName):
                  gaSettings = var

          gaSettingsParams = extractParamsFromVar(var)
          parameter = setGA4Params(parameter, eventName, measurementTagName, gaSettingsParams)
      
      if(parameter == "pageview"):
        ga4_tag['parameter'] = createGA4PVParams(measurementTagName)
      else:
        ga4_tag['parameter'] = list(filter(None, parameter))

      maxTagID += 1
      ga4_tags.append(ga4_tag)

  importJSON = json_data
  importJSON['containerVersion']['tag'] = ga4_tags
  
  return importJSON

In [ ]:
for i in iter(uploaded):
  importJSON = json.dumps(covertUAToGA4(i))
  
  with open('Import - {0}.json'.format(i.split("_")[0]), 'w') as f:
    f.write(importJSON)